### PPN (Post-Processing Network)

* Single zone nucleosynthesis code which solves coupled ODEs for for each isotope in your network

* As opposed to MESA (or other SE codes) and MPPNP there is only burn (**no mix**)

* Can be used to for a constant $T$, $\rho$ or trajectory

A **trajectory** follows $T$ and $\rho$ across time (from stellar model or other)

* Fast calculations, qualitatively similar results comapred to multi-zone. Useful for "first-order"
studies and nuclear rate uncertainty calculations. (Denissenkov et al 2014) 

* Can overestimate isotopes made at high $T$

### Setup
open a terminal and enter `ln -s /user /home/user/notebooks/`


### How to extract a trajectory from a stellar model (one way)

In [ ]:
#Load stellar data to select trajectory
%pylab nbagg
import sys
sys.path.insert(0,'/home/user/notebooks/user/NuGridSchool/NuGridPy')
from nugridpy2 import mesa as ms
from nugridpy2 import ascii_table as at

In [ ]:
M45 = ms.history_data('/data/ppm_rpod2/Stellar_models/PopIII/45M_led_mingap_letter_hdf/LOGS/')

In [ ]:
ifig=1;close(ifig);figure(ifig)
M45.kip_cont(ifig=1, mix_zones=50,burn_zones=50,xres=5000,yres=5000,ixaxis='age')

In [ ]:
at.writeTraj?

In [ ]:
#extract the data we need (age,T,rho)
idx = np.where((M45.get('star_age') > 0) & (M45.get('star_age') < 4e6))
age = M45.get('star_age')[idx][::4]
T_c = M45.get('log_center_T')[idx][::4]
Rho_c = M45.get('log_center_Rho')[idx][::4]

In [ ]:
#take a look at the data
ifig=2;close(ifig);figure(ifig)
plt.plot(age,T_c, 'r')
plt.plot(age,Rho_c,'b')
plt.ylim(0,8.5)
plt.ylabel(r'log T, $\rho$')
plt.xlabel(r'Age (yrs)')
plt.ticklabel_format(axis='x', style='sci',scilimits=(0,4))

In [ ]:
#need T and Rho not in log
T_c = 10**T_c/1e8
Rho_c = 10**Rho_c
data = [age,T_c,Rho_c]
print(len(T_c))

Note: If you want data from a specific mass (not central) coordinate you will have to loop over profiles

In [ ]:
#add this to a trajectory file
at.writeTraj(filename='trajectory.input', data=data, \
             ageunit=0, tunit=0, rhounit=0, idNum=0)

In [ ]:
%less trajectory.input

### Run the simulation
* Open the command line and enter the directory `/user/NuGridSchool`
* make your own directory here (VERY IMPORTANT!) `mkdir yourname` and enter it
* Make copy of the run directory PPN_PopIIIHburn `cp -r ./ppn-examples/PPN_PopIIIHburn PopIII_trajectory`
* Also run `cp -r ppn-examples/NPDATA .` from your user directory
* Enter your PopIII_trajectory directory
* Add Trajectory file to directory `cp /user/NuGridSchool/wendi-examples/Single-zone\ nucleosynthesis\ simulations/Summer_school_ppn_demo/trajectory.input .`
* Add initial abundance file to directory  
* Open ppn_frame.input and set T9 = 0. This is how the code knows to read the file named trajectory.input. Change IABUINI = 11 and change filename. Also turn flux option on.    

In [ ]:
from nugridpy2 import ppn

In [ ]:
p = ppn.abu_vector(sldir='/home/user/notebooks/Summer_school_ppn_demo/PopIII_trajectory')

In [ ]:
ifig=330;close(ifig);figure(ifig)
p.abu_chart(330,plotaxis=[0, 25, 0, 21])

In [ ]:
#Compare to mesa model

In [ ]:
M45p=ms.mesa_profile('/data/ppm_rpod2/Stellar_models/PopIII/45M_led_mingap_letter_hdf/LOGS',1634)

In [ ]:
ifig=4;close(ifig);figure(ifig)
M45p.plot('mass','h1', logy=True)
M45p.plot('mass','he4', logy=True)
M45p.plot('mass','c12', logy=True)
M45p.plot('mass','n14', logy=True)
M45p.plot('mass','o15', logy=True)
M45p.plot('mass','si28', logy=True)
plt.ylim(-15,0.2)
plt.xlim(0,15)

In [ ]:
ifig=330;close(ifig);figure(ifig)
%cd /user/NuGridSchool/yourdir/PopIII_trajectory/ 
p.abu_flux_chart(330)

In [ ]:
#Hot CNO as indicated by 13N(p,g)14O and 18O(p,g)19F

Let's change some rates -- maybe $^{18}\mathrm{O}(p,\gamma)^{19}F$ and $^{19}\mathrm{F}(p,\gamma)^{20}Ne$ by a factor of 1000 each (a lot! just for example)  

* Open ppn_physics.input change ININET to 1 and run ppn.exe
* Open your networksetup.txt file and find where these rates are used (set to T - not F) and multiply the rate by 10^3 (second to last column). Save and quit. 
* Open ppn_physics.input again change ININET to 3 and run ppn.exe


In [ ]:
ifig=330;close(ifig);figure(ifig)
p.abu_chart(330,plotaxis=[0, 25, 0, 21])